## Holds league wide statistcs

In [6]:
from enum import Enum
import pandas as pd

In [3]:
class LeagueStats:
    def __init__(self):
        self.average_starter_ppg: int
        self.median_starter_ppg: int
        self.average_starters: dict
        self.median_starters: dict
        self.average_best_bench: dict
        self.median_best_bench: dict

    def average_starter_ppg(self):
        return self.average_starter_ppg
        
    def set_average_starter_ppg(self, ppg):
        self.average_starter_ppg = ppg
    
    def median_starter_ppg(self):
        return self.median_starter_ppg
        
    def set_median_starter_ppg(self, ppg):
        self.median_starter_ppg = ppg

    # starter_position
    def average_starter_position_ppg(self, position):
        return self.average_starters[position.name]
        
    def set_average_starter_position_ppg(self, position, ppg):
        self.average_starters[position.name] = ppg
    
    def median_starter_position_ppg(self, position):
        return self.median_starters[position.name]
        
    def set_median_starter_position_ppg(self, position, ppg):
        self.median_starters[position.name] = ppg

    # bench_position
    def average_bench_position_ppg(self, position):
        return self.average_best_bench[position.name]
    
    def set_average_bench_position_ppg(self, position, ppg):
        self.average_best_bench[position.name] = ppg

    def median_bench_position_ppg(self, position):
        return self.median_best_bench[position.name]
    
    def set_median_bench_position_ppg(self, position, ppg):
        self.median_best_bench[position.name] = ppg

    # Boolean_position
    def average_position_ppg(self, position, starter=False, bench=False):
        if starter:
            return self.average_starter_position_ppg(position)
        if bench:
            return self.average_bench_position_ppg(position)
        raise Exception("Must specify either starter or bench")

    def median_position_ppg(self, position, starter=False, bench=False):
        if starter:
            return self.median_starter_position_ppg(position)
        if bench:
            return self.median_bench_position_ppg(position)
        raise Exception("Must specify either starter or bench")

    # Boolean_boolean
    def position_ppg(self, position, starter=False, bench=False, average=False, median=False):
        if average:
            return self.average_position_ppg(position, starter, bench)
        if median:
            return self.median_position_ppg(position, starter, bench)
        raise Exception("Must specify either average or median")

In [4]:
class POSITIONS(Enum):
    QB, WR, RB, TE, FLEX, SFLEX = range(6)

    def matched_positions(self):
        if self.name == 'FLEX':
            return ['WR','RB','TE']
        elif self.name == 'SFLEX':
            return ['QB','WR','RB','TE']
        return [self.name]

    def bench_count(self):
        bench_counts = {
            'RB': 2,
            'WR': 2,
            'TE': 1,
            'FLEX': 3,
            'SFLEX': 1,
            'QB': 0
        }
        return bench_counts[self.name]
        
    def starter_count(self):
        start_counts = {
            'RB': 1,
            'WR': 2,
            'TE': 1,
            'FLEX': 2,
            'SFLEX': 1,
            'QB': 1
        }
        return start_counts[self.name]

In [5]:
class Team:
    def __init__(self, owner):
        self.owner = owner
        self.contender_strength = 0
        # roster
        self.roster = pd.DataFrame()
        # roster, with starters by projection marked
        self.roster_starters = pd.DataFrame()

        self.starting_roster = {}
        
        self.scores = {
            'starter_score': 0,
            'starters': {},
            'bench': {}  
        }
    def get_starters(self):
        return self.roster_starters.query('Starter')

    def get_backups(self):
        return self.roster_starters.query('not Starter')

    # gets all players for position. position: String, count: Int (optional, if 0 returns all)
    def get_position_players(self, position, count=0, index=0):
        players = self.roster.query(f'position in {position.matched_positions()}')
        players = players.head(count) if count else players
    
    # gets all starters for position. position: String, count: Int (optional, if 0 returns all)
    def get_position_starters(self, position, count=0, index=-1):
        starters = self.starting_roster[position.name]
        starters = starters.head(count) if count else starters
        if index >= 0:
            if index >= starters.shape[0]:
                print(f'RETURNING NAN ROW for {self.owner} {position.name} starters index {index}')
                return pd.DataFrame(columns=starters.columns, index=range(1))
            starters = starters.iloc[index]
        return starters

    # gets all backups for position. position: String
    def get_position_backups(self, position, count=0, index=-1):
        backups = self.roster_starters.query(f'not Starter and position in {position.matched_positions()}')
        backups = backups.head(count) if count else backups
        if index >= 0:
            if index >= backups.shape[0]:
                print(f'RETURNING NAN ROW for {self.owner} {position.name} backups index {index}')
                return pd.DataFrame(columns=backups.columns, index=range(1))
            backups = backups.iloc[index]
        return backups